In [1]:
# Run this cell before continuing.
library(cowplot)
library(datateachr)
library(digest)
library(infer)
library(repr)
library(taxyvr)
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::lag()       masks stats::lag()
✖ lubridate::stamp() masks cowplot::stamp()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


# Examining Correlations between location and crime Rates

## Introduction
---

Our dataset contains crime data for 2022 in Vancouver. For each entry, we have recorded the type of crime, and we are specifically focusing on three crime types: "Break and Enter Residential/Other," "Break and Enter Commercial," and "Mischief." We also have information about the date and time the crime occurred, the address, neighborhood, and geographic coordinates.

The question we are trying to answer is whether there is a correlation between the geographic location of these specific types of crime and the frequency of criminal incidents. Our analysis seeks to determine if crime predominantly occurs in either the east or west side of Vancouver, as defined by the "X" coordinate. Crimes with "X" values less than 491017.47 are categorized as the "west" side, while values greater than or equal to this threshold are considered the "east" side.

In this study, we use the "X" coordinate as a key variable to understand the spatial distribution of these specific crime types. This approach allows us to focus on the geographical aspect of crime patterns for "Break and Enter Residential/Other," "Break and Enter Commercial," and "Mischief" incidents. 

Our null hypothesis is that these three specific types of crime are evenly distributed between the east and west sides of Vancouver. Our alternative hypothesis is that crime will occur more in the west side of Vancouver, X < 491017.47.0931, in coordinate. Downtown Vancouver has frequently been associated with higher rates of criminality, according to many news and these are one of the most hazardous locations in Vancouver.

### Parameters of Interest:

* Location Parameter (Mean): 

Our primary location parameter is the mean. We want to calculate the mean number of incidents for each of the three crime types on both the west and east sides of Vancouver. This parameter will help us identify if there is a significant difference in the average occurrence of these crimes in the two geographical areas.

* Scale Parameter (Standard Deviation):

The standard deviation will be a secondary parameter of interest. We'll calculate the standard deviation for each crime type in both west and east sides of Vancouver. This parameter is important because it allows us to assess the variability or dispersion of crime occurrences within each group. It can help us understand how consistent or variable crime patterns are in each area.

## Preliminary Results
---

In [2]:
# setwd("/home/jovyan/work/project")

In [5]:
crime_data <- read_csv("https://raw.githubusercontent.com/aradsab/Stat-201-Project/main/crimedata_csv_AllNeighbourhoods_2022.csv")
# cleaning the data
crime_data <- crime_data[complete.cases(crime_data), ]
crime_data <- subset(crime_data, X != 0)
crime_data <- crime_data %>%
  mutate(REGION = ifelse(X > 491017.47, "East", "West"))

head(crime_data)

Rows: 34281 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): TYPE, HUNDRED_BLOCK, NEIGHBOURHOOD
dbl (7): YEAR, MONTH, DAY, HOUR, MINUTE, X, Y

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y,REGION
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>
Break and Enter Commercial,2022,1,3,16,19,10XX ALBERNI ST,West End,491036.1,5459146,East
Break and Enter Commercial,2022,6,17,5,16,10XX ALBERNI ST,West End,491067.3,5459115,East
Break and Enter Commercial,2022,3,15,5,14,10XX ALBERNI ST,West End,491102.2,5459092,East
Break and Enter Commercial,2022,3,19,6,42,10XX ALBERNI ST,West End,491102.2,5459092,East
Break and Enter Commercial,2022,2,23,23,0,10XX BALFOUR AVE,Shaughnessy,490699.8,5455444,West
Break and Enter Commercial,2022,2,25,10,15,10XX BALFOUR AVE,Shaughnessy,490699.8,5455444,West


In [6]:
crime_X <- crime_data[c("REGION","TYPE","X")]
head(crime_X)

REGION,TYPE,X
<chr>,<chr>,<dbl>
East,Break and Enter Commercial,491036.1
East,Break and Enter Commercial,491067.3
East,Break and Enter Commercial,491102.2
East,Break and Enter Commercial,491102.2
West,Break and Enter Commercial,490699.8
West,Break and Enter Commercial,490699.8


In [9]:


crime_res <- crime_X[crime_X$TYPE == "Break and Enter Residential/Other", ]
crime_com <- crime_X[crime_X$TYPE == "Break and Enter Commercial", ]
crime_mis <- crime_X[crime_X$TYPE == "Mischief", ]

# one sample
sample_res <- crime_res %>%
    rep_sample_n(reps = 1, size = 100, replace = FALSE) %>% group_by(REGION) %>% summarise(total_crimes = n())

sample_com <- crime_com %>%
    rep_sample_n(reps = 1, size = 100, replace = FALSE) %>% group_by(REGION) %>% summarise(total_crimes = n())

sample_mis <- crime_mis %>%
    rep_sample_n(reps = 1, size = 100, replace = FALSE) %>% group_by(REGION) %>% summarise(total_crimes = n())

print(sample_res)
print(sample_com)
print(sample_mis)

# A tibble: 2 × 2
  REGION total_crimes
  <chr>         <int>
1 East             61
2 West             39
# A tibble: 2 × 2
  REGION total_crimes
  <chr>         <int>
1 East             65
2 West             35
# A tibble: 2 × 2
  REGION total_crimes
  <chr>         <int>
1 East             77
2 West             23


In [7]:
sd(crime_X_com_samples$mean)

[1] 327.8167

In [9]:
sd(crime_X_res_samples$mean)

[1] 566.9282

In [11]:
sd(crime_X_mis_samples$mean)

[1] 265.8403

Report: 

## Methods: Plan

---
For our hypothesis testing the null hypothesis is that there is equal crime rates between the west and east sides of Vancouver. The alternative is that the west side has more crime than the east. For this our confidence level will be 95%. In this case it would be better to use bootstrapping rather than the asymptotic method as the asymptotic method relies on the data to already follow a  normal distribution. If the p-value is less than 0.05 after conducting bootstrapping we will reject the null hypothesis.

* What do you expect to find?

We expect to find our alternative hypothesis to be true. Crime will be the most common in the west area (downtown), X < 491017.47 in coordinate, and this trend will stay consistent for different types of crimes and for different neighbourhoods within Vancouver. 
 
* What impact could such findings have?

Understanding the relationship between location and crime can tell law enforcement agencies when and where to allocate police resources efficiently. It may additionally cause more centred and efficient policing techniques, reducing crime rates during peak periods.

* What future questions could this lead to?

Beyond location, what different elements (i.e. weather, events, or tourism) could influence crime rates? Our research can be continued with those variables to inspect why the west side (downtown) has a significantly higher rate than other areas.

How do economic factors, such as income levels, education, and employment opportunities, differ between the west and east sides of Vancouver, and to what extent do these disparities correlate with crime rates.
References
Staysafevancouver. (2023, August 31). Vancouver crime rate: Areas to avoid. Stay Safe Vancouver. https://www.staysafevancouver.com/post/vancouver-crime-rate 

West side rated safer than East Side neighbourhoods: Vancouver police survey. The Georgia Straight. (2019, February 21). https://www.straight.com/news/1203621/west-side-rated-safer-east-side-neighbourhoods-vancouver-police-survey 
